Archivo utilizado para procesar la base de datos con la que trabajaremos

Importamos librerias

In [2]:
import pandas as pd
import numpy as np

Importamos Datos Privados

In [4]:
# Levanto la base de datos privados con la que trabajeremos
df_k = pd.read_csv('base_kavak.csv', sep=";")

# Observamos nuestro dataset
df_k.info()
df_k.shape
# (2532, 15)

# Armo una lista de los modelos disponibles en el dataset
modelos = df_k['Modelo'].unique().tolist()
# 146 modelos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2446 entries, 0 to 2445
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   2446 non-null   int64  
 1   price                2446 non-null   int64  
 2   currency             2446 non-null   object 
 3   condition            2446 non-null   object 
 4   Marca                2446 non-null   object 
 5   Puertas              2446 non-null   int64  
 6   Tipo de combustible  2446 non-null   object 
 7   Kilómetros           2238 non-null   float64
 8   Modelo               2446 non-null   object 
 9   Versión              2446 non-null   object 
 10  Año                  2446 non-null   int64  
 11  adressstate_name     2446 non-null   object 
 12  adresscity_name      2446 non-null   object 
 13  Motor                2446 non-null   float64
 14  Transmisión          2446 non-null   object 
dtypes: float64(2), int64(4), object(9)
mem

Importamos Datos Públicos

In [5]:
# Levanto la base de datos públicos con la que trabajeremos
df = pd.read_csv('data_publications.csv', sep=";")

# Observamos nuestro dataset
df.info()
df.shape
# (51360, 9)

# Evaluamos si tiene duplicados
print(df.duplicated().sum())
# 7624 filas duplicadas

# Sacamos los valores duplicados
df_new = df.drop_duplicates()
df_new.shape
# (43736, 9)

# Evaluamos si tiene nulos
df_new.isna().sum().sort_values()
# provincia      9

# Sacamos aquellas publicadas con valor en Dolares
df_new_ars = df_new.drop(df_new[df_new['currency']=='U$S'].index)

df_new_ars.shape
df_new_ars.info()
df_new_ars.describe()
# (35705, 9)

 
# Sumamos la columna modelo al data set público (a partir de los datos privados)
df_new_ars['modelo'] = pd.NaT
i = 0
for t in df_new_ars['title']:
    for m in modelos:
        if m.lower() in t.lower():
            df_new_ars['modelo'][i] = m
    i = i + 1


  
# Vemos la forma
df_new_ars.shape
# (35705, 10)


# Evaluo valores con nulos
df_new_ars.isna().sum().sort_values()
# provincia         8
# modelo         14711
 
# Eliminamos los valores con modelo nulo
df_new_ars = df_new_ars.drop(df_new_ars[df_new_ars['modelo'].isna()].index)
df_new_ars = df_new_ars.drop('title',axis=1)

df_new_ars.shape
# (20994, 9)


# Guardamos la información en un csv
# df_new_ars.to_csv('data_publications_process.csv',mode='a', sep=';', encoding='utf-8-sig', index=False)
# datos.to_excel('data_ready_to_model.csv',mode='a', sep=';', encoding='utf-8-sig', index=False)
#%%
# Exporto a un Excel
datos = df_new_ars
datos.to_excel('data_ready_to_model.xlsx', index=False)





################################################## Limpieza de datos##
######################################################################

# 1 - Sacamos los outliers

# Función que termina los outliers
def outlier_treatment(df,datacolumn):
    sorted(datacolumn)
    Q1,Q3 = np.percentile(datacolumn , [25,75])
    IQR = Q3 - Q1
    lower_range = Q1 - (1.5 * IQR)
    upper_range = Q3 + (1.5 * IQR)
    df.drop(df[(datacolumn < lower_range) | (datacolumn > upper_range)].index, inplace=True)
    return df

# Saco outlier KM
# datos = outlier_treatment(datos, datos.km)
# datos.shape
# (30808, 10)

# Saco outlier Year
# datos = outlier_treatment(datos, datos.year)
# datos.shape
# (30404, 10)

# Saco outlier Price
# datos = outlier_treatment(datos, datos.precio)
# datos.shape
# (28600, 10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51360 entries, 0 to 51359
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   precio       51360 non-null  int64 
 1   currency     51360 non-null  object
 2   localidad    51360 non-null  object
 3   provincia    51351 non-null  object
 4   km           51360 non-null  int64 
 5   year         51360 non-null  int64 
 6   marca        51360 non-null  object
 7   title        51360 non-null  object
 8   fecha_visto  51360 non-null  object
 9   id           51360 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 3.9+ MB
0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 38421 entries, 0 to 51359
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   precio       38421 non-null  int64 
 1   currency     38421 non-null  object
 2   localidad    38421 non-null  object
 3   provincia    38413 non-null 

C:\Users\LUCIA~1.LOP\AppData\Local\Temp/ipykernel_13248/2250355067.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new_ars['modelo'][i] = m
